## Preliminary - Install and Import Library

In [54]:
#!pip install pandas

In [55]:
#!pip install pyspark

In [56]:
import pandas as pd
import re
from datetime import datetime

In [57]:
from pyspark.sql import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

In [58]:
spark = SparkSession.builder.getOrCreate()

## Spark - 1 Load Data

In [59]:
# Start time
load_start = datetime.now()

In [60]:
listings = spark.read.format("com.databricks.spark.csv") \
                .option('header',True) \
                .option("inferSchema", "true")\
                .option("sep", ",")\
                .option('multiLine', True) \
                .option("escape",'\n') \
                .option("wholeFile","True") \
                .option("quoteAll","true") \
                .option("escape",'"') \
                .load('hdfs://localhost:9000/bdm/SINGAPORE/listings') \
                .withColumnRenamed("id", "listing_id")\
                .withColumn("price", expr("regexp_replace(price, '[$,]', '')").cast('float')) \
                .withColumn("neighborhood_overview", expr("regexp_replace(neighborhood_overview, '(<br />)|(<br/>)|(<b>)|(</b>)|(\n)','')")) \
                .withColumn("host_about", expr("regexp_replace(host_about, '(<br />)|(<br/>)|(<b>)|(</b>)|(\n)','')"))

In [61]:
listings.show(5)

+----------+--------------------+--------------+------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+---------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+------------+------------+---------+----------------+--------+----+--------------------+-----+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------

In [62]:
reviews = spark.read.format("com.databricks.spark.csv") \
                .option('header',True) \
                .option("inferSchema", "true")\
                .option("sep", ",")\
                .option('multiLine', True) \
                .option("escape",'\n') \
                .option("wholeFile","True") \
                .option("quoteAll","true") \
                .option("escape",'"') \
                .load('hdfs://localhost:9000/bdm/SINGAPORE/reviews') \
                .withColumn("comments", expr("regexp_replace(comments, '(<br />)|(<br/>)|(<b>)|(</b>)|(\n)','')")) \
                .withColumn("date", to_date(col("date"), "yyyy-MM-dd")) \
                .withColumn("review_year", year('date')) \
                .withColumn("review_month", month('date'))

In [63]:
reviews.show(5)

+----------+--------+----------+-----------+-------------+--------------------+-----------+------------+
|listing_id|      id|      date|reviewer_id|reviewer_name|            comments|review_year|review_month|
+----------+--------+----------+-----------+-------------+--------------------+-----------+------------+
|     50646|11909864|2014-04-18|    1356099|        James|A comfortable roo...|       2014|           4|
|     50646|13823948|2014-06-05|   15222393|        Welli|Stayed over at Su...|       2014|           6|
|     50646|15117222|2014-07-02|    5543172|        Cyril|It's been a lovel...|       2014|           7|
|     50646|15426462|2014-07-08|     817532|         Jake|We had a great ex...|       2014|           7|
|     50646|15552912|2014-07-11|   10942382|        Subba|Quiet condo. Comf...|       2014|           7|
+----------+--------+----------+-----------+-------------+--------------------+-----------+------------+
only showing top 5 rows



In [64]:
calendar = spark.read.format("com.databricks.spark.csv") \
                .option('header',True) \
                .option("inferSchema", "true")\
                .option("sep", ",")\
                .option('multiLine', True) \
                .option("escape",'\n') \
                .option("wholeFile","True") \
                .option("quoteAll","true") \
                .option("escape",'"') \
                .load('hdfs://localhost:9000/bdm/SINGAPORE/calendar') \
                .withColumn("price", expr("regexp_replace(price, '[$,]', '')").cast('float')) \
                .withColumn("adjusted_price", expr("regexp_replace(adjusted_price, '[$,]', '')").cast('float')) \
                .withColumn("date", to_date(col("date"), "yyyy-MM-dd")) \
                .withColumn("review_year", year('date')) \
                .withColumn("review_month", month('date'))

In [65]:
calendar.show(5)

+----------+----------+---------+-----+--------------+--------------+--------------+-----------+------------+
|listing_id|      date|available|price|adjusted_price|minimum_nights|maximum_nights|review_year|review_month|
+----------+----------+---------+-----+--------------+--------------+--------------+-----------+------------+
|   1833950|2021-10-28|        f|135.0|         135.0|             7|           365|       2021|          10|
|   4712676|2021-10-28|        f| 58.0|          58.0|             7|           365|       2021|          10|
|   4712676|2021-10-29|        f| 58.0|          58.0|             7|           365|       2021|          10|
|   4712676|2021-10-30|        f| 58.0|          58.0|             7|           365|       2021|          10|
|   4712676|2021-10-31|        f| 58.0|          58.0|             7|           365|       2021|          10|
+----------+----------+---------+-----+--------------+--------------+--------------+-----------+------------+
only showi

In [66]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-load_start).total_seconds()
print("Time taken to load data : {}".format(duration_load_time))

Time taken to load data : 5.97432


## Spark - 2 Query Data

### 2_query_data - Prepare data with listing id having more than 50 reviews

In [67]:
# Start time
query_start = datetime.now()

In [68]:
reviews_temp = reviews.select('listing_id', 'comments')
reviews_temp_groupby = reviews_temp.groupby('listing_id').count().withColumnRenamed('count','review_count')
reviews_filter50 = reviews_temp_groupby.filter(reviews_temp_groupby.review_count > 50)
listing_reviews = listings.select('listing_id', 'name', 'neighbourhood_cleansed', 'latitude', 'longitude', 'price', 
                                  'listing_url', 'review_scores_rating', 'review_scores_accuracy',
                                  'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                                  'review_scores_location', 'review_scores_value')

In [69]:
join_listing_50review = listing_reviews.join(reviews_filter50, 
                                             listing_reviews.listing_id == reviews_filter50.listing_id, 'inner')

join_listing_50review.select(listing_reviews.listing_id, listing_reviews.
                                                     review_scores_value, reviews_filter50.review_count).show(5)

+----------+-------------------+------------+
|listing_id|review_scores_value|review_count|
+----------+-------------------+------------+
|   1678744|               4.67|         150|
|   6300043|                4.8|          51|
|  28747912|               4.57|         103|
|   5908083|               4.36|          77|
|  11980197|               4.82|          71|
+----------+-------------------+------------+
only showing top 5 rows



In [70]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Prepare data with listing id having more than 50 reviews: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Prepare data with listing id having more than 50 reviews: 1.483511


In [71]:
print((join_listing_50review.count(), len(join_listing_50review.columns)))

(280, 16)


### 2_query_data_1 - [Overall Experience] Top 20 of listings in overall experience, with more than 50 reviews

In [72]:
# Start time
query_start = datetime.now()

In [73]:
top20_overall = join_listing_50review.sort(listing_reviews.review_scores_rating.desc())
top20_overall = top20_overall.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                                     listing_reviews.review_scores_rating, 
                                     reviews_filter50.review_count)
top20_overall = top20_overall.limit(20)

In [74]:
top20_overall.show()

+----------+--------------------+--------------------+------------+
|listing_id|         listing_url|review_scores_rating|review_count|
+----------+--------------------+--------------------+------------+
|  25435388|https://www.airbn...|                 5.0|          52|
|  33112080|https://www.airbn...|                4.96|          53|
|  35818699|https://www.airbn...|                4.95|          82|
|  16989989|https://www.airbn...|                4.94|          95|
|  31527262|https://www.airbn...|                4.94|         217|
|  22745179|https://www.airbn...|                4.93|         139|
|  20588444|https://www.airbn...|                4.93|          54|
|  20703034|https://www.airbn...|                4.93|          59|
|  23874999|https://www.airbn...|                4.93|          55|
|   2129215|https://www.airbn...|                4.93|         370|
|   9716208|https://www.airbn...|                4.93|          92|
|  30687199|https://www.airbn...|               

In [75]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 of listings in overall experience: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 of listings in overall experience: 1.625937


In [76]:
top20_overall.count()

20

### 2_query_data_2 -  [Cleanliness] Top 20 Listings Based on Review Score, with review number more than 50

In [77]:
# Start time
query_start = datetime.now()

In [78]:
top20_clean = join_listing_50review.sort(listing_reviews.review_scores_cleanliness.desc())

top20_clean = top20_clean.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                                 listing_reviews.review_scores_cleanliness, reviews_filter50.review_count)

top20_clean = top20_clean.limit(20)

In [79]:
top20_clean.show()

+----------+--------------------+-------------------------+------------+
|listing_id|         listing_url|review_scores_cleanliness|review_count|
+----------+--------------------+-------------------------+------------+
|   8264717|https://www.airbn...|                     4.97|         146|
|  15310627|https://www.airbn...|                     4.97|          65|
|  31114726|https://www.airbn...|                     4.96|          54|
|  23760158|https://www.airbn...|                     4.94|          89|
|  25435388|https://www.airbn...|                     4.94|          52|
|   8800763|https://www.airbn...|                     4.94|         215|
|  31527262|https://www.airbn...|                     4.94|         217|
|   9866917|https://www.airbn...|                     4.93|          70|
|  20588444|https://www.airbn...|                     4.93|          54|
|  25998215|https://www.airbn...|                     4.93|         169|
|  38134989|https://www.airbn...|                  

In [80]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Cleanliness: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Cleanliness: 0.770067


In [81]:
top20_clean.count()

20

### 2_query_data_3 - [Accuracy] Top 20 Listings Based on Review Score, with review number more than 50

In [82]:
# Start time
query_start = datetime.now()

In [83]:
top20_acc = join_listing_50review.sort(listing_reviews.review_scores_accuracy.desc())

top20_acc = top20_acc.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                             listing_reviews.review_scores_accuracy, reviews_filter50.review_count)

top20_acc = top20_acc.limit(20)

In [84]:
top20_acc.show()

+----------+--------------------+----------------------+------------+
|listing_id|         listing_url|review_scores_accuracy|review_count|
+----------+--------------------+----------------------+------------+
|  16989989|https://www.airbn...|                  4.99|          95|
|  25435388|https://www.airbn...|                  4.98|          52|
|  43379651|https://www.airbn...|                  4.97|          90|
|  31114726|https://www.airbn...|                  4.96|          54|
|  12614519|https://www.airbn...|                  4.96|          71|
|  30687199|https://www.airbn...|                  4.96|          99|
|  22745179|https://www.airbn...|                  4.95|         139|
|  23874999|https://www.airbn...|                  4.95|          55|
|  20703034|https://www.airbn...|                  4.95|          59|
|  11381088|https://www.airbn...|                  4.95|         164|
|  35818699|https://www.airbn...|                  4.94|          82|
|  11860123|https://

In [85]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Accuracy: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Accuracy: 0.857835


In [86]:
top20_acc.count()

20

### 2_query_data_4 - [Check-in] Top 20 Listings Based on Review Score, with review number more than 50

In [87]:
# Start time
query_start = datetime.now()

In [88]:
top20_checkin = join_listing_50review.sort(listing_reviews.review_scores_checkin.desc())

top20_checkin = top20_checkin.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                             listing_reviews.review_scores_checkin, reviews_filter50.review_count)

top20_checkin = top20_checkin.limit(20)

In [89]:
top20_checkin.show()

+----------+--------------------+---------------------+------------+
|listing_id|         listing_url|review_scores_checkin|review_count|
+----------+--------------------+---------------------+------------+
|   9928008|https://www.airbn...|                  5.0|          92|
|  30687199|https://www.airbn...|                  5.0|          99|
|  15310627|https://www.airbn...|                  5.0|          65|
|  21063861|https://www.airbn...|                 4.98|          81|
|   3877859|https://www.airbn...|                 4.98|          63|
|  38134989|https://www.airbn...|                 4.98|          56|
|  23874999|https://www.airbn...|                 4.98|          55|
|   6718219|https://www.airbn...|                 4.98|          58|
|  21887155|https://www.airbn...|                 4.98|          90|
|  31201327|https://www.airbn...|                 4.98|          59|
|   5376240|https://www.airbn...|                 4.97|         240|
|   9866917|https://www.airbn...| 

In [90]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Check-in: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Check-in: 0.678651


### 2_query_data_5 - [Communication with host] Top 20 Listings Based on Review Score, with review number more than 50

In [91]:
# Start time
query_start = datetime.now()

In [92]:
top20_comm = join_listing_50review.sort(listing_reviews.review_scores_communication.desc())

top20_comm = top20_comm.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                             listing_reviews.review_scores_communication, reviews_filter50.review_count)

top20_comm = top20_comm.limit(20)

In [93]:
top20_comm.show()

+----------+--------------------+---------------------------+------------+
|listing_id|         listing_url|review_scores_communication|review_count|
+----------+--------------------+---------------------------+------------+
|  11980197|https://www.airbn...|                        5.0|          71|
|  23874999|https://www.airbn...|                        5.0|          55|
|  31201327|https://www.airbn...|                        5.0|          59|
|   1302185|https://www.airbn...|                       4.99|          91|
|  30687199|https://www.airbn...|                       4.99|          99|
|  35818699|https://www.airbn...|                       4.98|          82|
|   7605283|https://www.airbn...|                       4.98|         322|
|  25435388|https://www.airbn...|                       4.98|          52|
|  16989989|https://www.airbn...|                       4.98|          95|
|  38134989|https://www.airbn...|                       4.98|          56|
|  14223124|https://www.a

In [94]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Communication with Host: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Communication with Host: 1.121634


### 2_query_data_6 - [Location] Top 20 Listings Based on Review Score, with review number more than 50

In [95]:
# Start time
query_start = datetime.now()

In [96]:
top20_location = join_listing_50review.sort(listing_reviews.review_scores_location.desc())

top20_location = top20_location.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                             listing_reviews.review_scores_location, reviews_filter50.review_count)

top20_location = top20_location.limit(20)

In [97]:
top20_location.show()

+----------+--------------------+----------------------+------------+
|listing_id|         listing_url|review_scores_location|review_count|
+----------+--------------------+----------------------+------------+
|  23446208|https://www.airbn...|                  4.99|         116|
|  15792137|https://www.airbn...|                  4.99|         105|
|  11863888|https://www.airbn...|                  4.99|          75|
|  25892078|https://www.airbn...|                  4.99|          78|
|  23874999|https://www.airbn...|                  4.98|          55|
|  11817134|https://www.airbn...|                  4.97|         131|
|  15310627|https://www.airbn...|                  4.97|          65|
|  24276647|https://www.airbn...|                  4.97|          73|
|  20588444|https://www.airbn...|                  4.96|          54|
|  11381088|https://www.airbn...|                  4.96|         164|
|  17928170|https://www.airbn...|                  4.96|          94|
|  11865360|https://

In [98]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Location: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Location: 0.971124


### 2_query_data_7 - [Value] Top 20 Listings Based on Review Score, with review number more than 50

In [99]:
# Start time
query_start = datetime.now()

In [100]:
top20_value = join_listing_50review.sort(listing_reviews.review_scores_value.desc())

top20_value = top20_value.select(listing_reviews.listing_id, listing_reviews.listing_url, 
                             listing_reviews.review_scores_value, reviews_filter50.review_count)

top20_value = top20_value.limit(20)

In [101]:
top20_value.show()

+----------+--------------------+-------------------+------------+
|listing_id|         listing_url|review_scores_value|review_count|
+----------+--------------------+-------------------+------------+
|   9866917|https://www.airbn...|               4.94|          70|
|  23874999|https://www.airbn...|               4.91|          55|
|   9716208|https://www.airbn...|               4.91|          92|
|  25435388|https://www.airbn...|                4.9|          52|
|  16231781|https://www.airbn...|                4.9|         127|
|  12614519|https://www.airbn...|                4.9|          71|
|  25998215|https://www.airbn...|               4.88|         169|
|   7605283|https://www.airbn...|               4.87|         322|
|   5798211|https://www.airbn...|               4.87|         151|
|   6630341|https://www.airbn...|               4.86|          98|
|   6619955|https://www.airbn...|               4.86|          64|
|  16231741|https://www.airbn...|               4.86|         

In [102]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-query_start).total_seconds()
print("Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Value: {}".format(duration_load_time))

Time taken to run Part 2 Query Data_Top 20 Listings Based on Review Score_Value: 0.876288


## Spark - 3 Analysis Data

### 3_analysis_data_1 - Listing - Count by Room Type

In [103]:
# Start time
analysis_start = datetime.now()

In [104]:
roomtype = listings.select('listing_id', 'room_type')
count_roomtype = roomtype.groupby('room_type').count().withColumnRenamed('count','cnt')
count_roomtype = count_roomtype.sort(count_roomtype.cnt.desc())

In [105]:
count_roomtype.show()

+---------------+----+
|      room_type| cnt|
+---------------+----+
|   Private room|2013|
|Entire home/apt|1769|
|     Hotel room| 182|
|    Shared room| 159|
+---------------+----+



In [106]:
# Write the file to HDFS
count_roomtype.write.csv('hdfs://localhost:9000/bdm/spark/count_roomtype')

In [107]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Count by Room Type: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Count by Room Type: 1.517129


### 3_analysis_data_2 - Listing - Count by Property Type

In [108]:
# Start time
analysis_start = datetime.now()

In [109]:
propertytype = listings.select('listing_id', 'property_type')
count_propertytype = propertytype.groupby('property_type').count().withColumnRenamed('count','cnt')
count_propertytype = count_propertytype.sort(count_propertytype.cnt.desc())

In [110]:
count_propertytype.show(truncate=False)

+-----------------------------------+---+
|property_type                      |cnt|
+-----------------------------------+---+
|Private room in rental unit        |831|
|Entire condominium (condo)         |657|
|Entire rental unit                 |582|
|Entire serviced apartment          |428|
|Private room in condominium (condo)|341|
|Room in boutique hotel             |239|
|Private room in residential home   |187|
|Room in hotel                      |138|
|Private room in townhouse          |113|
|Room in hostel                     |76 |
|Shared room in hostel              |61 |
|Entire residential home            |42 |
|Private room in hostel             |36 |
|Private room in serviced apartment |35 |
|Private room in bungalow           |35 |
|Shared room in rental unit         |31 |
|Room in serviced apartment         |30 |
|Private room in bed and breakfast  |28 |
|Shared room in bed and breakfast   |27 |
|Private room                       |26 |
+---------------------------------

In [111]:
# Write the file to HDFS
count_propertytype.write.csv('hdfs://localhost:9000/bdm/spark/count_propertytype')

In [112]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Count by Property Type: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Count by Property Type: 1.231272


### 3_analysis_data_3 - Top 10 Property Type with Expensive Average Price

In [113]:
# Start time
analysis_start = datetime.now()

In [114]:
price_property = listings.select('listing_id', 'price', 'property_type')
avgprice_property = price_property.groupby('property_type').avg('price') \
                    .withColumnRenamed('avg(price)','avg_price')
avgprice_property = avgprice_property.sort(avgprice_property.avg_price.desc())
top10_avgprice_property = avgprice_property.limit(10)

In [115]:
top10_avgprice_property.show(truncate=False)

+---------------------------+------------------+
|property_type              |avg_price         |
+---------------------------+------------------+
|Entire villa               |1016.3333333333334|
|Boat                       |849.4             |
|Entire place               |592.0             |
|Entire townhouse           |526.2             |
|Entire residential home    |470.6190476190476 |
|Room in hotel              |329.89130434782606|
|Shared room in bungalow    |307.5             |
|Private room in guest suite|298.125           |
|Entire serviced apartment  |289.20093457943926|
|Tent                       |277.0             |
+---------------------------+------------------+



In [116]:
# Write the file to HDFS
top10_avgprice_property.write.csv('hdfs://localhost:9000/bdm/spark/top10_avgprice_property')

In [117]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Top 10 Property Type with Expensive Average Price: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Top 10 Property Type with Expensive Average Price: 1.40381


### 3_analysis_data_4 - Top 10 Property Type with Cheapest Average Price

In [118]:
# Start time
analysis_start = datetime.now()

In [119]:
price_property = listings.select('listing_id', 'price', 'property_type')
avgprice_property = price_property.groupby('property_type').avg('price') \
                    .withColumnRenamed('avg(price)','avg_price')
avgprice_property = avgprice_property.sort(avgprice_property.avg_price.asc())
bot10_avgprice_property = avgprice_property.limit(10)

In [120]:
bot10_avgprice_property.show(truncate=False)

+--------------------------------+------------------+
|property_type                   |avg_price         |
+--------------------------------+------------------+
|Shared room in guesthouse       |25.0              |
|Shared room in residential home |36.5              |
|Shared room                     |41.7              |
|Private room in casa particular |44.0              |
|Private room in lighthouse      |45.0              |
|Shared room in villa            |45.0              |
|Shared room in bed and breakfast|50.666666666666664|
|Entire home/apt                 |58.333333333333336|
|Shared room in hostel           |62.31147540983606 |
|Room in hostel                  |66.3157894736842  |
+--------------------------------+------------------+



In [121]:
# Write the file to HDFS
bot10_avgprice_property.write.csv('hdfs://localhost:9000/bdm/spark/bot10_avgprice_property')

In [122]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Top 10 Property Type with Cheapest Average Price: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Top 10 Property Type with Cheapest Average Price: 1.158784


### 3_analysis_data_5 - Summary for Number of Reviews Comment by Year

In [123]:
# Start time
analysis_start = datetime.now()

In [124]:
group_reviews_yr = reviews.groupby('review_year').count().withColumnRenamed('count','cnt')
group_reviews_yr = group_reviews_yr.sort(reviews.review_year.desc())

In [125]:
group_reviews_yr.show()

+-----------+-----+
|review_year|  cnt|
+-----------+-----+
|       2021| 3216|
|       2020| 6649|
|       2019|14792|
|       2018|10020|
|       2017| 7392|
|       2016| 4319|
|       2015| 2127|
|       2014|  482|
|       2013|  222|
|       2012|   98|
|       2011|   11|
+-----------+-----+



In [126]:
# Write the file to HDFS
group_reviews_yr.write.csv('hdfs://localhost:9000/bdm/spark/group_reviews_yr')

In [127]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Summary for Number of Reviews Comment by Year: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Summary for Number of Reviews Comment by Year: 2.833815


### 3_analysis_data_6 - Average Price by Location (Neighbourhood_cleansed)

In [128]:
# Start time
analysis_start = datetime.now()

In [129]:
neighborhood = listings.select('listing_id', 'neighbourhood_cleansed', 'price')
neighborhood_price = neighborhood.groupby('neighbourhood_cleansed').avg('price') \
                                        .withColumnRenamed('avg(price)','avg_price')
neighborhood_price = neighborhood_price.sort(neighborhood_price.avg_price.desc())

In [130]:
neighborhood_price.show()

+----------------------+------------------+
|neighbourhood_cleansed|         avg_price|
+----------------------+------------------+
|                  Tuas|           10286.0|
|      Southern Islands|1830.5263157894738|
|               Orchard| 568.6666666666666|
|          Marina South|             359.0|
|             Pasir Ris| 332.4166666666667|
|          Lim Chu Kang|             324.0|
|               Pioneer|             305.0|
|          Sungei Kadut|             290.0|
|               Tanglin| 268.7391304347826|
|                Newton| 254.6315789473684|
|       Singapore River| 251.9181818181818|
|              Clementi|230.02409638554218|
|  Central Water Cat...|             227.5|
|             Toa Payoh|225.89743589743588|
|          River Valley| 217.6139240506329|
|               Hougang| 210.2325581395349|
|                Novena|193.06140350877192|
|         Downtown Core|185.69716088328076|
|                Rochor|171.74358974358975|
|            Queenstown| 171.321

In [131]:
# Write the file to HDFS
neighborhood_price.write.csv('hdfs://localhost:9000/bdm/spark/neighborhood_price')

In [132]:
# Duration of Execution Time - Loading Data
duration_load_time = (datetime.now()-analysis_start).total_seconds()
print("Time taken to run Part 3 Analysis Data_Average Price by Location: {}".format(duration_load_time))

Time taken to run Part 3 Analysis Data_Average Price by Location: 1.314195
